<a href="https://colab.research.google.com/github/NielsRogge/Transformers-Tutorials/blob/master/BERT/Custom_Named_Entity_Recognition_with_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Fine-tuning BERT for named-entity recognition**

In this notebook, we are going to use **BertForTokenClassification** which is included in the [Transformers library](https://github.com/huggingface/transformers) by HuggingFace. This model has BERT as its base architecture, with a token classification head on top, allowing it to make predictions at the token level, rather than the sequence level. Named entity recognition is typically treated as a token classification problem, so that's what we are going to use it for.

This tutorial uses the idea of **transfer learning**, i.e. first pretraining a large neural network in an unsupervised way, and then fine-tuning that neural network on a task of interest. In this case, BERT is a neural network pretrained on 2 tasks: masked language modeling and next sentence prediction. Now, we are going to fine-tune this network on a NER dataset. Fine-tuning is supervised learning, so this means we will need a labeled dataset.

If you want to know more about BERT, I suggest the following resources:
* the original [paper](https://arxiv.org/abs/1810.04805)
* Jay Allamar's [blog post](http://jalammar.github.io/illustrated-bert/) as well as his [tutorial](http://jalammar.github.io/a-visual-guide-to-using-bert-for-the-first-time/)
* Chris Mccormick's [Youtube channel](https://www.youtube.com/channel/UCoRX98PLOsaN8PtekB9kWrw)
* Abbishek Kumar Mishra's [Youtube channel](https://www.youtube.com/user/abhisheksvnit)

The following notebook largely follows the same structure as the tutorials by Abhishek Kumar Mishra. For his tutorials on the Transformers library, see his [Github repository](https://github.com/abhimishra91/transformers-tutorials).

NOTE: this notebook assumes basic knowledge about deep learning, BERT, and native PyTorch. If you want to learn more Python, deep learning and PyTorch, I highly recommend cs231n by Stanford University and the FastAI course by Jeremy Howard et al. Both are freely available on the web.  

Now, let's move on to the real stuff!

#### **Importing Python Libraries and preparing the environment**

This notebook assumes that you have the following libraries installed:
* pandas
* numpy
* sklearn
* pytorch
* transformers
* seqeval

As we are running this in Google Colab, the only libraries we need to additionally install are transformers and seqeval (GPU version):

In [1]:
!pip install transformers seqeval[gpu]

In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertConfig, BertForTokenClassification

As deep learning can be accellerated a lot using a GPU instead of a CPU, make sure you can run this notebook in a GPU runtime (which Google Colab provides for free! - check "Runtime" - "Change runtime type" - and set the hardware accelerator to "GPU").

We can set the default device to GPU using the following code (if it prints "cuda", it means the GPU has been recognized):

In [3]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cuda


#### **Downloading and preprocessing the data**
Named entity recognition (NER) uses a specific annotation scheme, which is defined (at least for European languages) at the *word* level. An annotation scheme that is widely used is called **[IOB-tagging](https://en.wikipedia.org/wiki/Inside%E2%80%93outside%E2%80%93beginning_(tagging))**, which stands for Inside-Outside-Beginning. Each tag indicates whether the corresponding word is *inside*, *outside* or at the *beginning* of a specific named entity. The reason this is used is because named entities usually comprise more than 1 word. 

Let's have a look at an example. If you have a sentence like "Barack Obama was born in Hawaï", then the corresponding tags would be   [B-PERS, I-PERS, O, O, O, B-GEO]. B-PERS means that the word "Barack" is the beginning of a person, I-PERS means that the word "Obama" is inside a person, "O" means that the word "was" is outside a named entity, and so on. So one typically has as many tags as there are words in a sentence.

So if you want to train a deep learning model for NER, it requires that you have your data in this IOB format (or similar formats such as [BILOU](https://stackoverflow.com/questions/17116446/what-do-the-bilou-tags-mean-in-named-entity-recognition)). There exist many annotation tools which let you create these kind of annotations automatically (such as Spacy's [Prodigy](https://prodi.gy/), [Tagtog](https://docs.tagtog.net/) or [Doccano](https://github.com/doccano/doccano)). You can also use Spacy's [biluo_tags_from_offsets](https://spacy.io/api/goldparse#biluo_tags_from_offsets) function to convert annotations at the character level to IOB format.

Here, we will use a NER dataset from [Kaggle](https://www.kaggle.com/namanj27/ner-dataset) that is already in IOB format. One has to go to this web page, download the dataset, unzip it, and upload the csv file to this notebook. Let's print out the first few rows of this csv file:

In [10]:
data = pd.read_csv("macrostart_ner/ner_dataset.csv")
print(len(data))
data.head()

64911


,sentence,word_labels
0,the Nolichucky Shale (aquitard) and the karsti...,"O,B-strat_name,I-strat_name,O,O,O,O,O,O,O,O,O,O,O"
1,"and a Plainfield sand, deficient in P, were tr...","O,O,B-strat_name,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O..."
2,", Weaver rhyolite; 3, trachytic and keratophyr...","O,O,O,O,O,O,O,O,O,O,O,O,B-lith,O,O"
3,siltstone gy sandy siltstone gn-gy fine sandst...,"B-strat_name,O,B-strat_name,I-strat_name,O,B-s..."
4,". (1998) Plains sill, Prichard Fm. 1469 ± 3 Ma...","O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O"


In [19]:
unique_terms = set([term for row in data['word_labels'].str.split(',') for term in row])
label2id = {k: v for v, k in enumerate(unique_terms)}
id2label = {v: k for v, k in enumerate(unique_terms)}
print(label2id, id2label)

{'B-strat_name': 0, 'I-strat_name': 1, 'I-lith': 2, 'B-lith_att': 3, 'I-lith_att': 4, 'B-lith': 5, 'O': 6} {0: 'B-strat_name', 1: 'I-strat_name', 2: 'I-lith', 3: 'B-lith_att', 4: 'I-lith_att', 5: 'B-lith', 6: 'O'}


Let's verify that a random sentence and its corresponding tags are correct:

In [20]:
data.iloc[41].sentence

'-coast boreal forests (Montmorency, Rocky Harbour, and Gander). Certainly, decomposition'

In [21]:
data.iloc[41].word_labels

'O,O,O,O,O,O,O,O,O,O'

#### **Preparing the dataset and dataloader**

Now that our data is preprocessed, we can turn it into PyTorch tensors such that we can provide it to the model. Let's start by defining some key variables that will be used later on in the training/evaluation process:

In [13]:
MAX_LEN = 128
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 2
EPOCHS = 1
LEARNING_RATE = 1e-05
MAX_GRAD_NORM = 10
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

A tricky part of NER with BERT is that BERT relies on **wordpiece tokenization**, rather than word tokenization. This means that we should also define the labels at the wordpiece-level, rather than the word-level! 

For example, if you have word like "Washington" which is labeled as "b-gpe", but it gets tokenized to "Wash", "##ing", "##ton", then we will have to propagate the word’s original label to all of its wordpieces: "b-gpe", "b-gpe", "b-gpe". The model should be able to produce the correct labels for each individual wordpiece. The function below (taken from [here](https://github.com/chambliss/Multilingual_NER/blob/master/python/utils/main_utils.py#L118)) implements this.






In [22]:
def tokenize_and_preserve_labels(sentence, text_labels, tokenizer):
    """
    Word piece tokenization makes it difficult to match word labels
    back up with individual word pieces. This function tokenizes each
    word one at a time so that it is easier to preserve the correct
    label for each subword. It is, of course, a bit slower in processing
    time, but it will help our model achieve higher accuracy.
    """

    tokenized_sentence = []
    labels = []

    sentence = sentence.strip()

    for word, label in zip(sentence.split(), text_labels.split(",")):

        # Tokenize the word and count # of subwords the word is broken into
        tokenized_word = tokenizer.tokenize(word)
        n_subwords = len(tokenized_word)

        # Add the tokenized word to the final tokenized word list
        tokenized_sentence.extend(tokenized_word)

        # Add the same label to the new list of labels `n_subwords` times
        labels.extend([label] * n_subwords)

    return tokenized_sentence, labels

Note that this is a **design decision**. You could also decide to only label the first wordpiece of each word and let the model only learn this (this is what was done in the original BERT paper, see Github discussion [here](https://github.com/huggingface/transformers/issues/64#issuecomment-443703063)). Another design decision could be to give the first wordpiece of each word the original word label, and then use the label “X” for all subsequent subwords of that word.

All of them lead to good performance.

Next, we define a regular PyTorch [dataset class](https://pytorch.org/docs/stable/data.html) (which transforms examples of a dataframe to PyTorch tensors). Here, each sentence gets tokenized, the special tokens that BERT expects are added, the tokens are padded or truncated based on the max length of the model, the attention mask is created and the labels are created based on the dictionary which we defined above. 

For more information about BERT's inputs, see [here](https://huggingface.co/transformers/glossary.html).  

In [23]:
class dataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __getitem__(self, index):
        # step 1: tokenize (and adapt corresponding labels)
        sentence = self.data.sentence[index]  
        word_labels = self.data.word_labels[index]  
        tokenized_sentence, labels = tokenize_and_preserve_labels(sentence, word_labels, self.tokenizer)
        
        # step 2: add special tokens (and corresponding labels)
        tokenized_sentence = ["[CLS]"] + tokenized_sentence + ["[SEP]"] # add special tokens
        labels.insert(0, "O") # add outside label for [CLS] token
        labels.insert(-1, "O") # add outside label for [SEP] token

        # step 3: truncating/padding
        maxlen = self.max_len

        if (len(tokenized_sentence) > maxlen):
          # truncate
          tokenized_sentence = tokenized_sentence[:maxlen]
          labels = labels[:maxlen]
        else:
          # pad
          tokenized_sentence = tokenized_sentence + ['[PAD]'for _ in range(maxlen - len(tokenized_sentence))]
          labels = labels + ["O" for _ in range(maxlen - len(labels))]

        # step 4: obtain the attention mask
        attn_mask = [1 if tok != '[PAD]' else 0 for tok in tokenized_sentence]
        
        # step 5: convert tokens to input ids
        ids = self.tokenizer.convert_tokens_to_ids(tokenized_sentence)

        label_ids = [label2id[label] for label in labels]
        # the following line is deprecated
        #label_ids = [label if label != 0 else -100 for label in label_ids]
        
        return {
              'ids': torch.tensor(ids, dtype=torch.long),
              'mask': torch.tensor(attn_mask, dtype=torch.long),
              #'token_type_ids': torch.tensor(token_ids, dtype=torch.long),
              'targets': torch.tensor(label_ids, dtype=torch.long)
        } 
    
    def __len__(self):
        return self.len

Now, based on the class we defined above, we can create 2 datasets, one for training and one for testing. Let's use a 80/20 split:

In [24]:
train_size = 0.8
train_dataset = data.sample(frac=train_size,random_state=200)
test_dataset = data.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)

print("FULL Dataset: {}".format(data.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

training_set = dataset(train_dataset, tokenizer, MAX_LEN)
testing_set = dataset(test_dataset, tokenizer, MAX_LEN)

FULL Dataset: (64911, 2)
TRAIN Dataset: (51929, 2)
TEST Dataset: (12982, 2)


Let's have a look at the first training example:

In [25]:
training_set[0]

{'ids': tensor([  101,  1045,  1066,  1011, 10722, 28140,  2078,  1011,  1027,  1054,
          7842,  1050, 16525,  4886,  1066,  1046,  1035,  1066,  1066,  6053,
          2937,  1066, 26794,  1064,  2019,  1066,  1016,  1056,  1000,  1000,
          1056,  1005,  1015,  1012,  1066, 24471,  2063,  1045,  1024,  1079,
          1011,  1000, 16844,  4718,  1012,  1011,  1011,  1528, 10731,  9759,
          2884,   102,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,  

Let's verify that the input ids and corresponding targets are correct:

In [26]:
training_set[0]["ids"]

tensor([  101,  1045,  1066,  1011, 10722, 28140,  2078,  1011,  1027,  1054,
         7842,  1050, 16525,  4886,  1066,  1046,  1035,  1066,  1066,  6053,
         2937,  1066, 26794,  1064,  2019,  1066,  1016,  1056,  1000,  1000,
         1056,  1005,  1015,  1012,  1066, 24471,  2063,  1045,  1024,  1079,
         1011,  1000, 16844,  4718,  1012,  1011,  1011,  1528, 10731,  9759,
         2884,   102,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0])

In [53]:
# print the first 30 tokens and corresponding labels
for token, label in zip(tokenizer.convert_ids_to_tokens(training_set[0]["ids"][:30]), training_set[0]["targets"][:30]):
  print('{0:10}  {1}'.format(token, id2label[label.item()]))

[CLS]       O
za          B-per
##hee       B-per
##r         B-per
khan        I-per
was         O
mar         O
-           O
93          O
for         O
india       B-geo
.           O
[SEP]       O
[PAD]       O
[PAD]       O
[PAD]       O
[PAD]       O
[PAD]       O
[PAD]       O
[PAD]       O
[PAD]       O
[PAD]       O
[PAD]       O
[PAD]       O
[PAD]       O
[PAD]       O
[PAD]       O
[PAD]       O
[PAD]       O
[PAD]       O


Now, let's define the corresponding PyTorch dataloaders:

In [27]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

#### **Defining the model**

Here we define the model, BertForTokenClassification, and load it with the pretrained weights of "bert-base-uncased". The only thing we need to additionally specify is the number of labels (as this will determine the architecture of the classification head).

Note that only the base layers are initialized with the pretrained weights. The token classification head of top has just randomly initialized weights, which we will train, together with the pretrained weights, using our labelled dataset. This is also printed as a warning when you run the code cell below.

Then, we move the model to the GPU.

In [28]:
model = BertForTokenClassification.from_pretrained('bert-base-uncased', 
                                                   num_labels=len(id2label),
                                                   id2label=id2label,
                                                   label2id=label2id)
model.to(device)

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-u

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

#### **Training the model**

Before training the model, let's perform a sanity check, which I learned thanks to Andrej Karpathy's wonderful [cs231n course](http://cs231n.stanford.edu/) at Stanford (see also his [blog post about debugging neural networks](http://karpathy.github.io/2019/04/25/recipe/)). The initial loss of your model should be close to -ln(1/number of classes) = -ln(1/17) = 2.83. 

Why? Because we are using cross entropy loss. The cross entropy loss is defined as -ln(probability score of the model for the correct class). In the beginning, the weights are random, so the probability distribution for all of the classes for a given token will be uniform, meaning that the probability for the correct class will be near 1/17. The loss for a given token will thus be -ln(1/17). As PyTorch's [CrossEntropyLoss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html) (which is used by `BertForTokenClassification`) uses *mean reduction* by default, it will compute the mean loss for each of the tokens in the sequence (in other words, for all of the 512 tokens). The mean of 512 times -log(1/17) is, you guessed it, -log(1/17).  

Let's verify this:



In [29]:
ids = training_set[0]["ids"].unsqueeze(0)
mask = training_set[0]["mask"].unsqueeze(0)
targets = training_set[0]["targets"].unsqueeze(0)
ids = ids.to(device)
mask = mask.to(device)
targets = targets.to(device)
outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
initial_loss = outputs[0]
initial_loss

tensor(2.4223, device='cuda:0', grad_fn=<NllLossBackward>)

This looks good. Let's also verify that the logits of the neural network have a shape of (batch_size, sequence_length, num_labels):

In [30]:
tr_logits = outputs[1]
tr_logits.shape

torch.Size([1, 128, 7])

Next, we define the optimizer. Here, we are just going to use Adam with a default learning rate. One can also decide to use more advanced ones such as AdamW (Adam with weight decay fix), which is [included](https://huggingface.co/transformers/main_classes/optimizer_schedules.html) in the Transformers repository, and a learning rate scheduler, but we are not going to do that here.

In [31]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)

Now let's define a regular PyTorch training function. It is partly based on [a really good repository about multilingual NER](https://github.com/chambliss/Multilingual_NER/blob/master/python/utils/main_utils.py#L344).

In [32]:
# Defining the training function on the 80% of the dataset for tuning the bert model
def train(epoch):
    tr_loss, tr_accuracy = 0, 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []
    # put model in training mode
    model.train()
    
    for idx, batch in enumerate(training_loader):
        
        ids = batch['ids'].to(device, dtype = torch.long)
        mask = batch['mask'].to(device, dtype = torch.long)
        targets = batch['targets'].to(device, dtype = torch.long)

        outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
        loss, tr_logits = outputs.loss, outputs.logits
        tr_loss += loss.item()

        nb_tr_steps += 1
        nb_tr_examples += targets.size(0)
        
        if idx % 100==0:
            loss_step = tr_loss/nb_tr_steps
            print(f"Training loss per 100 training steps: {loss_step}")
           
        # compute training accuracy
        flattened_targets = targets.view(-1) # shape (batch_size * seq_len,)
        active_logits = tr_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        # now, use mask to determine where we should compare predictions with targets (includes [CLS] and [SEP] token predictions)
        active_accuracy = mask.view(-1) == 1 # active accuracy is also of shape (batch_size * seq_len,)
        targets = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)
        
        tr_preds.extend(predictions)
        tr_labels.extend(targets)
        
        tmp_tr_accuracy = accuracy_score(targets.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy
    
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=MAX_GRAD_NORM
        )
        
        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    print(f"Training loss epoch: {epoch_loss}")
    print(f"Training accuracy epoch: {tr_accuracy}")

And let's train the model!

In [33]:
for epoch in range(EPOCHS):
    print(f"Training epoch: {epoch + 1}")
    train(epoch)

Training epoch: 1
Training loss per 100 training steps: 2.0629823207855225
Training loss per 100 training steps: 0.6435352636150794
Training loss per 100 training steps: 0.5168080055224362
Training loss per 100 training steps: 0.46401060415908346
Training loss per 100 training steps: 0.42458620381325557
Training loss per 100 training steps: 0.40076417768905737
Training loss per 100 training steps: 0.383198232809934
Training loss per 100 training steps: 0.3737659420015638
Training loss per 100 training steps: 0.3596988261285495
Training loss per 100 training steps: 0.35003472091040916
Training loss per 100 training steps: 0.34430577899966686
Training loss per 100 training steps: 0.33679118095055477
Training loss per 100 training steps: 0.3299901493011565
Training loss per 100 training steps: 0.3251498528483462
Training loss per 100 training steps: 0.31859048375385224
Training loss per 100 training steps: 0.31396425893898805
Training loss per 100 training steps: 0.3073982141102239
Traini

#### **Evaluating the model**

Now that we've trained our model, we can evaluate its performance on the held-out test set (which is 20% of the data). Note that here, no gradient updates are performed, the model just outputs its logits. 

In [34]:
def valid(model, testing_loader):
    # put model in evaluation mode
    model.eval()
    
    eval_loss, eval_accuracy = 0, 0
    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels = [], []
    
    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):
            
            ids = batch['ids'].to(device, dtype = torch.long)
            mask = batch['mask'].to(device, dtype = torch.long)
            targets = batch['targets'].to(device, dtype = torch.long)
            
            outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
            loss, eval_logits = outputs.loss, outputs.logits
            
            eval_loss += loss.item()

            nb_eval_steps += 1
            nb_eval_examples += targets.size(0)
        
            if idx % 100==0:
                loss_step = eval_loss/nb_eval_steps
                print(f"Validation loss per 100 evaluation steps: {loss_step}")
              
            # compute evaluation accuracy
            flattened_targets = targets.view(-1) # shape (batch_size * seq_len,)
            active_logits = eval_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
            # now, use mask to determine where we should compare predictions with targets (includes [CLS] and [SEP] token predictions)
            active_accuracy = mask.view(-1) == 1 # active accuracy is also of shape (batch_size * seq_len,)
            targets = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)
            
            eval_labels.extend(targets)
            eval_preds.extend(predictions)
            
            tmp_eval_accuracy = accuracy_score(targets.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy
    
    #print(eval_labels)
    #print(eval_preds)

    labels = [id2label[id.item()] for id in eval_labels]
    predictions = [id2label[id.item()] for id in eval_preds]

    #print(labels)
    #print(predictions)
    
    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps
    print(f"Validation Loss: {eval_loss}")
    print(f"Validation Accuracy: {eval_accuracy}")

    return labels, predictions

As we can see below, performance is quite good! Accuracy on the test test is > 93%.

In [35]:
labels, predictions = valid(model, testing_loader)

Validation loss per 100 evaluation steps: 0.0795961394906044
Validation loss per 100 evaluation steps: 0.14639085290162815
Validation loss per 100 evaluation steps: 0.13935074146630808
Validation loss per 100 evaluation steps: 0.1297567404626383
Validation loss per 100 evaluation steps: 0.12759746954441906
Validation loss per 100 evaluation steps: 0.1275802714408128
Validation loss per 100 evaluation steps: 0.1240972865771149
Validation loss per 100 evaluation steps: 0.12427562851536508
Validation loss per 100 evaluation steps: 0.12436714797898704
Validation loss per 100 evaluation steps: 0.12658249338123365
Validation loss per 100 evaluation steps: 0.12558035616396362
Validation loss per 100 evaluation steps: 0.12258903884261654
Validation loss per 100 evaluation steps: 0.12470708326243685
Validation loss per 100 evaluation steps: 0.125737570284937
Validation loss per 100 evaluation steps: 0.12655873688221705
Validation loss per 100 evaluation steps: 0.12613235027988445
Validation los

However, the accuracy metric is misleading, as a lot of labels are "outside" (O), even after omitting predictions on the [PAD] tokens. What is important is looking at the precision, recall and f1-score of the individual tags. For this, we use the seqeval Python library: 

In [46]:
from seqeval.metrics import classification_report

print(classification_report([labels], [predictions]))

              precision    recall  f1-score   support

        lith       0.79      0.76      0.78      2741
    lith_att       0.75      0.83      0.79      5217
  strat_name       0.74      0.75      0.74     21482

   micro avg       0.75      0.76      0.76     29440
   macro avg       0.76      0.78      0.77     29440
weighted avg       0.75      0.76      0.76     29440



In [47]:
save_dir = "macrostart_ner_model"
model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)

('macrostart_ner_model/tokenizer_config.json',
 'macrostart_ner_model/special_tokens_map.json',
 'macrostart_ner_model/vocab.txt',
 'macrostart_ner_model/added_tokens.json')

In [54]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline

model_name = "macrostart_ner_model"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)

In [55]:
ner_pipeline = pipeline(task="ner", model=model.to("cpu"), tokenizer=tokenizer)

In [95]:
import json
import os

def move_to_upper(curr_level, curr_hiearchy, level_hierarchy):
    if curr_level not in curr_hiearchy:
        return
    
    hierachy_idx = level_hierarchy.index(curr_level) - 1
    for term in curr_hiearchy[curr_level]:
        # Determine the highest level
        term_h_idx = hierachy_idx
        term_idx = curr_hiearchy[curr_level][term]["index"]
        while term_h_idx > 0 and level_hierarchy[term_h_idx] not in curr_hiearchy:
            term_h_idx -= 1

        # Determine the distance to move to
        move_h_name = level_hierarchy[term_h_idx]
        matched_word, matched_distance = None, None
        for upper_term in curr_hiearchy[move_h_name]:
            upper_idx = curr_hiearchy[move_h_name][upper_term]["index"]
            upper_distance = abs(upper_idx - term_idx)
            if matched_distance is None or upper_distance < matched_distance:
                matched_distance = upper_distance
                matched_word = upper_term
        
        # Move the current term
        if curr_level not in curr_hiearchy[move_h_name][matched_word]:
            curr_hiearchy[move_h_name][matched_word][curr_level] = []
        
        curr_hiearchy[move_h_name][matched_word][curr_level].append(curr_hiearchy[curr_level][term])

def extract_kg(paragraph, save_path):
    prediction = ner_pipeline(paragraph)
    
    terms_by_label = {}
    curr_idx = 0
    while curr_idx < len(prediction):
        word = "" 
        prev_token_idx = prediction[curr_idx]['start']
        word_label = prediction[curr_idx]["entity"].replace("B-", "").replace("I-", "").strip()
        start_idx = prediction[curr_idx]["index"]
        while curr_idx < len(prediction) and prediction[curr_idx]['start'] == prev_token_idx:
            word += str(prediction[curr_idx]["word"]).replace("##", "").strip()
            prev_token_idx = prediction[curr_idx]["end"]
            curr_idx += 1
        
        if word_label not in terms_by_label:
            terms_by_label[word_label] = {}
        terms_by_label[word_label][word] = {"index" : start_idx, "term" : word}
    
    merged_terms = {}
    for label in terms_by_label:
        label_words = list(terms_by_label[label].items())
        label_words.sort(key = lambda x : x[1]["index"])
        print("Processing", label, "with terms", label_words)

        merged_terms[label] = {}
        word_idx = 0
        while word_idx < len(label_words):
            term_idx = label_words[word_idx][1]["index"]
            prev_term_idx = term_idx - 1
            term = ""
            while word_idx < len(label_words) and label_words[word_idx][1]["index"] == (prev_term_idx + 1):
                term += " " + label_words[word_idx][1]["term"]
                prev_term_idx = label_words[word_idx][1]["index"]
                word_idx += 1
            term = term.strip()

            merged_terms[label][term] = {"index" : term_idx, "term" : term}
    

    '''
    level_hierarchy = ["strat_name", "lith", "lith_att"]
    for idx in range(len(level_hierarchy) - 1, 0, -1):
        move_to_upper(level_hierarchy[idx], merged_terms, level_hierarchy)
    '''
    
    result = {
        "paragraph" : paragraph,
        "entities" : terms_by_label
    }
    print("Saving to", save_path)
    with open(save_path, 'w+', encoding='utf-8') as writer:
        json.dump(result, writer, ensure_ascii=False, indent=4)    

In [96]:
dataset_dir = "manual_datasets"
for file_name in os.listdir(dataset_dir):
    if "json" not in file_name or file_name[0] == '.' or "prediction" in file_name:
        continue
    
    with open(os.path.join(dataset_dir, file_name), 'r') as reader:
        paragraph = json.load(reader)["paragraph"]
    
    save_path = os.path.join(dataset_dir, file_name.replace("example", "example_prediction"))
    print("Running for example", file_name)
    extract_kg(paragraph, save_path)
    break

Running for example example_1.json
Processing strat_name with terms [('rhyolite', {'index': 27, 'term': 'rhyolite'}), ('tuff', {'index': 31, 'term': 'tuff'}), ('volcanic', {'index': 205, 'term': 'volcanic'}), ('mount', {'index': 312, 'term': 'mount'}), ('galen', {'index': 313, 'term': 'galen'}), ('volcanics', {'index': 314, 'term': 'volcanics'}), ('igneous', {'index': 342, 'term': 'igneous'})]
Processing lith with terms [('dacite', {'index': 24, 'term': 'dacite'}), ('andesite', {'index': 52, 'term': 'andesite'}), ('sedimentary', {'index': 175, 'term': 'sedimentary'})]
Processing lith_att with terms [('basaltic', {'index': 250, 'term': 'basaltic'})]
Saving to manual_datasets/example_prediction_1.json
